In [1]:
import pandas as pd
import pyfolio
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

ModuleNotFoundError: No module named 'pyfolio'

In [ ]:
def get_daily_return(df):
    df['daily_return']=df.account_value.pct_change(1)
    #df=df.dropna()
    print('Sharpe: ',(252**0.5)*df['daily_return'].mean()/ df['daily_return'].std())
    return df

In [10]:
def backtest_strat(df):
    strategy_ret= df.copy()
    strategy_ret['Date'] = pd.to_datetime(strategy_ret['Date'])
    strategy_ret.set_index('Date', drop = False, inplace = True)
    strategy_ret.index = strategy_ret.index.tz_localize('UTC')
    del strategy_ret['Date']
    ts = pd.Series(strategy_ret['daily_return'].values, index=strategy_ret.index)
    return ts

array([[3, 4, 5],
       [6, 7, 8],
       [0, 1, 2]])

In [5]:
def get_account_value(model_name):
    df_account_value=pd.DataFrame()
    for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
        temp = pd.read_csv('results/secondRun_RR/account_value_trade_{}_{}.csv'.format(model_name,i))
        df_account_value = df_account_value.append(temp,ignore_index=True)
    df_account_value = pd.DataFrame({'account_value':df_account_value['0']})
    sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
    print(sharpe)
    df_account_value=df_account_value.join(df_trade_date[63:].reset_index(drop=True))
    return df_account_value

array([6, 7, 8])

In [8]:
df=pd.read_csv('data/dow_30_2009_2020.csv')
rebalance_window = 63
validation_window = 63
unique_trade_date = df[(df.datadate > 20151001)&(df.datadate <= 20200707)].datadate.unique()
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

array([2, 1, 0], dtype=int32)

In [11]:
extensemble_acct_val = get_account_value('ensemble')
ppo_acct_val = get_account_value('PPO')
a2c_acct_val = get_account_value('A2C')
ddpg_acct_val = get_account_value('DDPG')
sac_acct_val = get_account_value('SAC')

array([[2, 1, 0],
       [5, 4, 3],
       [8, 7, 6]])

In [ ]:
extensemble_acct_val = get_daily_return(extensemble_acct_val)
ppo_acct_val = get_daily_return(ppo_acct_val)
a2c_acct_val = get_daily_return(a2c_acct_val)
ddpg_acct_val = get_daily_return(ddpg_acct_val)
sac_acct_val = get_daily_return(sac_acct_val)

In [ ]:
extensemble_acct_val['Date'] = test_dji['Date']
ppo_acct_val['Date'] = test_dji['Date']
a2c_acct_val['Date'] = test_dji['Date']
ddpg_acct_val['Date'] = test_dji['Date']
sac_acct_val['Date'] = test_dji['Date']

In [ ]:
ppo_acct_val.head()

In [ ]:
extensemble_strat = backtest_strat(extensemble_acct_val[0:1097])
ppo_strat = backtest_strat(ppo_acct_val[0:1097])
a2c_strat = backtest_strat(a2c_acct_val[0:1097])
ddpg_strat = backtest_strat(ddpg_acct_val[0:1097])
sac_strat = backtest_strat(sac_acct_val[0:1097])

In [ ]:
# extensemble_acct_val.to_csv('results/df_account_value_extensemble_daily_return.csv',index=False)
# ppo_acct_val.to_csv('results/df_account_value_ppo_daily_return.csv',index=False)
# a2c_acct_val.to_csv('results/df_account_value_a2c_daily_return.csv',index=False)
# ddpg_acct_val.to_csv('results/df_account_value_ddpg_daily_return.csv',index=False)
# sac_acct_val.to_csv('results/df_account_value_sac_daily_return.csv',index=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = extensemble_strat,
                                   benchmark_rets=dow_strat, set_context=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = ppo_strat,
                                   benchmark_rets=dow_strat, set_context=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = a2c_strat,
                                   benchmark_rets=dow_strat, set_context=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = ddpg_strat,
                                   benchmark_rets=dow_strat, set_context=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = sac_strat,
                                   benchmark_rets=dow_strat, set_context=False)

In [ ]:
extensemble_cum_ret = pyfolio.timeseries.cum_returns(extensemble_strat)
ppo_cum_ret = pyfolio.timeseries.cum_returns(ppo_strat)
a2c_cum_ret = pyfolio.timeseries.cum_returns(a2c_strat)
ddpg_cum_ret = pyfolio.timeseries.cum_returns(ddpg_strat)
sac_cum_ret = pyfolio.timeseries.cum_returns(sac_strat)
dow_cum_ret = pyfolio.timeseries.cum_returns(dow_strat)

In [ ]:
extensemble_cum_ret.name = "Extended Ensemble Method"
ppo_cum_ret.name = "PPO"
a2c_cum_ret.name = "A2C"
ddpg_cum_ret.name = "DDPG"
sac_cum_ret.name = "SAC"
dow_cum_ret.name = "DJIA"
df_cum_ret = pd.concat([extensemble_cum_ret, ppo_cum_ret, a2c_cum_ret, ddpg_cum_ret, sac_cum_ret, dow_cum_ret], axis = 1)

In [ ]:
df_cum_ret.plot(figsize = (18, 6), title = "Cumulative Return with Transaction Cost", grid = True)
matplotlib.pyplot.ylabel("Cumulative Return")
matplotlib.pyplot.show()